# Introduction 


## How to use this notebook
This example runs Sima in locally based on input created in the [Create load case folders with unique Sima input](#loadcase_id) section. The code is only a pilot and only intended for testing. 

This note bookshould be run in the following way:
1. [Installation](#installation) can be run to install all the relevant tools, like python modules and local worker executable
2. [Initialize Workflow](#initialize), run this Python code once at every notebook start to set up basic settings. [Set up custom user parameters](#custom) section should be changed if you want to change workspace or switch between cloud and local run. 2. If you changed something in this section, remember to rerun  [Set up OneWorkflow client](#builder).
3. [Run analysis](#run) shall be run each time a new Sima analysis needs to be run.
4. [Run Wasim and Sestra](#runwasim) run Wasim and Sestra using results from above Sima run.

### Postprocessing
It is now possible to read the SIN/FEM file using SifIO, please consult the [Python examples](https://test.pypi.org/project/dnv-sifio/) C# [documentation](https://sesam.dnv.com/dev/api/sifio/). This notebook provides a small example script for getting node coordinates and displacements for the 200 first nodes for loadcase 11, [postprocessing.py](Workspace/CommonFiles/postprocessing.py). The script will be run just after Wasim and Sestra for each load case, see the [Run Wasim and Sestra](#runwasim) section. The section [Post processing](#postprocessing) shows how the post processed result files may be read and visualized in this notebook. More Python examples will be provided later. Relevant additional documents are:
- The input interface file documentations can be found [here](https://sesam.dnv.com/download/windows64/sesam_input_interface_format.pdf)
- The results interface format [here](https://sesam.dnv.com/download/windows64/sesam_results_interface_format.pdf)

The Python package should be installed with this command : pip install -i https://test.pypi.org/simple/ dnv-sifio --user as done in the [Installation](#installation) section.

# Initialize workflow <a id='initialize'></a>
Run only once when notebook is opened.

## Set up custom user parameters <a id='custom'></a>

In [1]:
from pathlib import Path
import os

oneWorlflowTMPFolder = r'c:\oneworkflowTmp' #due to possible issues with long file paths we prefer to have this folder at the root
if not os.path.exists(oneWorlflowTMPFolder):
    try:
        print("Trying to create tmp folder for one workflow local execution")
        os.mkdir(oneWorlflowTMPFolder)
    except:
        print("did not manage to create tmp folder for local execution. Check that you have privileges to create it or try to manually create it from the coomand line.")

# local workspace, all results will be put here after local or cloud runs
# location of common files for all analysis, has to be below workspacePath
root_folder = os.getcwd()
print(root_folder)
workspacePath = Path(root_folder, 'Workspace')
workspaceId = "SE28"
loadcase_file = f"{workspacePath}\\test_cases.xlsx"
wasim_input_file = "test_cases_wasim_input.xlsx"
stask_file = "SimaTemplate.stask"
cloud_run = False
notebook_root_folder = os.getcwd()

c:\Users\kblu\source\repos\improveflowGIT\SE28ExampleSimaWasimSestra


## Set up OneWorkflow client <a id='builder'></a>
Run only once workbook is started or if some parameters above are changed.

In [2]:
from oneWorkflowToolBox import *
workflow_client = one_workflow_client(
    workspaceId, workspacePath, cloud_run, tmp=oneWorlflowTMPFolder) 
workspace = workflow_client.one_workflow_config.workspace_config
commonfiles_folder = workspace.common_files_directory
results_folder = workspace.results_directory
#If running locally the code below will also start the local workflow host.
if (cloud_run):
    workflow_client.login()


The temporary blob storage directory is: c:\OneWorkflowTemp\oc_2xp69qpj_blob
The temporary jobs root directory is: c:\OneWorkflowTemp\oc_j4oq3d63_jobs
c:\OneWorkflowTemp


## Upload common files for the job <a id='upload'></a>
This step uploads all common files in folder *commonFilesDirectory*  to the job. Only needed to run if new common files are to be uploaded or workspace changed.

In [3]:

from dnv.onecompute.directory_client import FileOptions
try:
    workflow_client.upload_common_files(FileOptions(
        # max_size_bytes=124_000,
        #patterns=["**/*.py","**/*.inp"],
        overwrite=True))
except Exception as e:
    print(e)
    print("Ignore this error message if the files are already present.")


Uploading c:\Users\kblu\source\repos\improveflowGIT\SE28ExampleSimaWasimSestra\Workspace\CommonFiles\copyFiles.py to c:\OneWorkflowTemp\oc_2xp69qpj_blob\SE28\CommonFiles\copyFiles.py
Uploading c:\Users\kblu\source\repos\improveflowGIT\SE28ExampleSimaWasimSestra\Workspace\CommonFiles\Floater.ssg to c:\OneWorkflowTemp\oc_2xp69qpj_blob\SE28\CommonFiles\Floater.ssg
Uploading c:\Users\kblu\source\repos\improveflowGIT\SE28ExampleSimaWasimSestra\Workspace\CommonFiles\postprocessing.py to c:\OneWorkflowTemp\oc_2xp69qpj_blob\SE28\CommonFiles\postprocessing.py
Uploading c:\Users\kblu\source\repos\improveflowGIT\SE28ExampleSimaWasimSestra\Workspace\CommonFiles\requirements.txt to c:\OneWorkflowTemp\oc_2xp69qpj_blob\SE28\CommonFiles\requirements.txt
Uploading c:\Users\kblu\source\repos\improveflowGIT\SE28ExampleSimaWasimSestra\Workspace\CommonFiles\sestra_template.inp to c:\OneWorkflowTemp\oc_2xp69qpj_blob\SE28\CommonFiles\sestra_template.inp
Uploading c:\Users\kblu\source\repos\improveflowGIT\SE2

# Run Sima analysis <a id='run'></a>
This code will fetch data from the blob storage created in the step above, and run all the job tasks. The code will wait for all tasks to complete before downloading the results.

In [4]:
from oneWorkflowToolBox import run_workflow_async
from SimaHelper import *
import json
"""Tests SIMA and Python commands"""
# Upload Input Files
workflow_client.upload_input_files()

#Sima path must be specified
sima_settings = SimaSettings(sima_exe_path=r'C:\Program Files\DNV\Sima V4.4-00')
sima_work_unit = SimaTaskCreator(sima_settings, workflow_client).get_sima_work_unit(loadcase_file, stask_file)

# Create Parallel Work Unit and Job
job = workflow_client.create_job(sima_work_unit)
job_json = json.dumps(job, default=lambda o: o.encode(), indent=4)

#print(job_json)
# Run workflow
downloadOptions = FileOptions(
    max_size_bytes=11124_000,
    patterns=["**/*.txt", "**/*.lis", "**/*.mlg"])
await run_workflow_async(job, workflow_client, downloadOptions)


Error: load cases directory c:\Users\kblu\source\repos\improveflowGIT\SE28ExampleSimaWasimSestra\Workspace\LoadCases does not exist.
The work item e4187b73-d776-4fdf-964c-63af042c35f6 message is 'Created'
The work item test001 message is 'Created'
The work item test002 message is 'Created'
The work item test003 message is 'Created'
The progress of the job is '0%'. The message is ''
The work item test001 message is ''
The work item test002 message is ''
The work item test003 message is ''
The work item test001 message is '2023-08-31 15:58:51,306 [main] INFO  SIMALog - Setting log level to ALL - at no.marintek.sima.util.SIMALog.setLogLevel(SIMALog.java:51)'
The work item test002 message is '2023-08-31 15:58:51,308 [main] INFO  SIMALog - Setting log level to ALL - at no.marintek.sima.util.SIMALog.setLogLevel(SIMALog.java:51)'
The work item test003 message is '2023-08-31 15:58:51,853 [main] INFO  SIMALog - Setting log level to ALL - at no.marintek.sima.util.SIMALog.setLogLevel(SIMALog.java

# Run Wasim and Sestra <a id='runwasim'></a>

In [5]:
from dnv.oneworkflow import PythonCommand, CompositeExecutableCommand
from dnv.sesam.wasim_command import WasimStruCommand,  WasimSetupCommand
from dnv.onecompute.flowmodel import WorkUnit, SchedulingOptions, FailureStrategy
from SesamUtilities import WasimAndSestraTaskCreator
import pandas as pd
from dnv.oneworkflow import  ParallelWork
from oneWorkflowToolBox import Platform
import json
import os
from dnv.sesam.sestra_command import SestraCommand
workspace.results_directory = "Results"
load_cases = ["test001", "test002"]

os.chdir(workspacePath)
topSuperElement = 3
# due to 5 field width on Sestra cards we need to use as short name here
additionalTemplateParameters = {
    'FMT': topSuperElement, 'topsel': topSuperElement}

def run_wasim_and_sestra_using_results_from_sima(
        results_directory: str, load_cases: str) -> ParallelWork:
    """Creates a parallel work unit"""
    df_cases = pd.read_excel(os.path.join(
        workspacePath, wasim_input_file), index_col=0)
    parallel_work_units = list[WorkUnit]()

    for casename, case in df_cases.iterrows():
        if not casename in load_cases:
            print("skipping " + casename)
            continue
        
        load_case_result_files_dir = os.path.join(results_directory, casename)
        casedict = case.to_dict()
        
        cmd = WasimAndSestraTaskCreator(
            load_case_result_files_dir, commonfiles_folder, casedict, additionalTemplateParameters).CreateTasks()
        work_unit = (
            WorkUnit(cmd, f"post_rerun_{casename}")
            .input_directory(load_case_result_files_dir)
            .output_directory(load_case_result_files_dir, include_files=["**/sima.*", "**/*.txt", "**/*.tda", "**/*.bin", "**/*.log", "**/*.inp", "**/*.lis", "**/*.mlg", "**/*.sin"])
        )
        parallel_work_units.append(work_unit)

    return ParallelWork(parallel_work_units)
print(os.getcwd())
print(workflow_client.workspace_path)

work_unit = run_wasim_and_sestra_using_results_from_sima(
    workspace.results_directory, load_cases)
if not cloud_run:
    workflow_client.upload_common_files()
    workflow_client.upload_result_files()
print(workflow_client.temp_path)


downloadOptions = FileOptions(
    min_size_bytes =0,
    max_size_bytes=11124_000,
    patterns=["**/*.txt", "**/*.lis", "**/*.mlg", "**/*.sin"])
await run_workflow_async(workflow_client.create_job(work_unit), workflow_client, downloadOptions)



c:\Users\kblu\source\repos\improveflowGIT\SE28ExampleSimaWasimSestra\Workspace
c:\Users\kblu\source\repos\improveflowGIT\SE28ExampleSimaWasimSestra\Workspace
skipping test003
Uploading c:\Users\kblu\source\repos\improveflowGIT\SE28ExampleSimaWasimSestra\Workspace\CommonFiles\copyFiles.py to c:\OneWorkflowTemp\oc_2xp69qpj_blob\SE28\CommonFiles\copyFiles.py
Uploading c:\Users\kblu\source\repos\improveflowGIT\SE28ExampleSimaWasimSestra\Workspace\CommonFiles\Floater.ssg to c:\OneWorkflowTemp\oc_2xp69qpj_blob\SE28\CommonFiles\Floater.ssg
Uploading c:\Users\kblu\source\repos\improveflowGIT\SE28ExampleSimaWasimSestra\Workspace\CommonFiles\postprocessing.py to c:\OneWorkflowTemp\oc_2xp69qpj_blob\SE28\CommonFiles\postprocessing.py
Uploading c:\Users\kblu\source\repos\improveflowGIT\SE28ExampleSimaWasimSestra\Workspace\CommonFiles\requirements.txt to c:\OneWorkflowTemp\oc_2xp69qpj_blob\SE28\CommonFiles\requirements.txt
Uploading c:\Users\kblu\source\repos\improveflowGIT\SE28ExampleSimaWasimSestr

# Post processing  <a id='postprocessing'></a>

In [6]:
from IPython.display import display
import numpy as np
import pandas as pd
import glob
from ipywidgets import interactive

lc = 11
dataFrames = {}
print(workspacePath)
for folder in glob.glob(f"{workspacePath}\\Results\\*"):
    test_name = folder.split("\\")[-1]
    try:
        data = np.loadtxt(f"{folder}\\postprocessedresultsLC{lc}.txt")
        dispdata = {
            "x-coord": data[:, 0],
            "total-disp":  np.sqrt(data[:, 3]**2+data[:, 4]**2+data[:, 5]**2)
        }
        dataFrames[test_name] = pd.DataFrame(dispdata)
    except:
        print("Failed loading data for test :" + test_name)

def multiplot(resultcase):
    dataFrames[resultcase].plot(
        x="x-coord", y="total-disp", kind="scatter", figsize=(15, 15))


interactive_plot = interactive(multiplot, resultcase=dataFrames.keys())
interactive_plot


c:\Users\kblu\source\repos\improveflowGIT\SE28ExampleSimaWasimSestra\Workspace
Failed loading data for test :test003


interactive(children=(Dropdown(description='resultcase', options=('test001', 'test002'), value='test001'), Out…